In [ ]:
!pip install -q \
streamlit \
pyngrok \
langchain \
langchain-community \
langchain-openai \
langchain-text-splitters \
openai \
faiss-cpu \
pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 135.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/3

In [ ]:
%%writefile app.py
import streamlit as st

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

# ----------------------------------
# Streamlit Config
# ----------------------------------
st.set_page_config(page_title="Resume RAG Chatbot", layout="wide")
st.title("📄 Resume RAG Chatbot")
st.write("Upload multiple resume PDFs and ask questions about candidates.")

# ----------------------------------
# Upload PDFs
# ----------------------------------
uploaded_files = st.file_uploader(
    "Upload Resume PDFs",
    type=["pdf"],
    accept_multiple_files=True
)

# ----------------------------------
# Vector Store
# ----------------------------------
@st.cache_resource
def create_vector_db(files):
    documents = []

    for file in files:
        path = f"/tmp/{file.name}"
        with open(path, "wb") as f:
            f.write(file.getbuffer())

        loader = PyPDFLoader(path)
        pages = loader.load()

        for page in pages:
            page.metadata["source"] = file.name
            documents.append(page)

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )

    chunks = splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    return FAISS.from_documents(chunks, embeddings)

# ----------------------------------
# RAG Pipeline
# ----------------------------------
if uploaded_files:
    db = create_vector_db(uploaded_files)
    st.success("✅ Resumes indexed successfully")

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=db.as_retriever(search_kwargs={"k": 5}),
        return_source_documents=True
    )

    query = st.text_input("Ask a question about the candidates")

    if query:
        with st.spinner("Thinking..."):
            result = qa.invoke({"query": query})

        st.subheader("🧠 Answer")
        st.write(result["result"])

        st.subheader("📌 Source Resumes")
        for doc in result["source_documents"]:
            st.write(f"• {doc.metadata['source']}")


Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
# Make sure you have pyngrok installed
# !pip install pyngrok

from pyngrok import ngrok
from google.colab import userdata

# It's recommended to store your ngrok authtoken as a Colab Secret
# Go to the '🔑' icon on the left panel, click 'Add a new secret',
# name it 'NGROK_AUTH_TOKEN', and paste your authtoken as the value.
# Then, ensure 'Notebook access' is enabled for this secret.

# Retrieve the authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('RAG')

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("ngrok authtoken set successfully!")
else:
    print("Error: NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")

ngrok authtoken set successfully!


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("🔗 Streamlit URL:", public_url)


🔗 Streamlit URL: NgrokTunnel: "https://unlogistical-uncriticizing-eryn.ngrok-free.dev" -> "http://localhost:8501"
